In [1]:
!pip install "transformers>4.45.2" datasets accelerate bitsandbytes trl peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 24.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import timeit
model_path = "./base_model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm)

In [4]:

messages = [
  {"role": "system", "content": "너는 AI 어시스턴트야."},
  {"role": "user", "content": "오늘 날씨 어때?"}
]

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

eos_token_id = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

start_time = timeit.default_timer()

outputs = model.generate(
  inputs,
  eos_token_id=eos_token_id,
  max_length=4096
)

print(tokenizer.decode(outputs[0]))

print(f"ttft : {timeit.default_timer() - start_time}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

너는 AI 어시스턴트야.<|eot_id|><|start_header_id|>user<|end_header_id|>

오늘 날씨 어때?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

죄송합니다. 저는 실시간 정보를 제공할 수 없는 AI입니다. 현재 날씨 상황을 확인하려면 실시간 날씨 업데이트를 제공하는 앱이나 웹사이트를 참조해 주시기 바랍니다. 

하지만 일반적으로 날씨 예보를 요청하신다면, 제가 대략적인 계절별 날씨 패턴에 대해 설명해드릴 수는 있습니다. 

예를 들어, 봄에는 대체로 온화한 날씨가 이어지지만 가끔씩은 갑작스러운 봄비가 올 수 있습니다. 여름에는 무더운 날씨가 주를 이루며, 가을에는 서늘해지면서 가을비가 내릴 수 있습니다. 겨울에는 추운 날씨와 함께 눈이 내리는 경우가 많습니다.

하지만 이는 일반적인 패턴일 뿐, 지역마다 차이가 있을 수 있음을 유의하시기 바랍니다. 정확한 날씨 정보를 원하신다면 전문적인 기상 정보 제공 서비스를 이용하시는 것이 좋습니다.<|eot_id|>
ttft : 20.714480007067323
